In [1]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 15.9 MB/s 


In [3]:
#Imports and libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import pandas as pd
import os
import cv2
import seaborn as sn
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [4]:
#Directory of train and test folders
train_path = "/content/drive/MyDrive/SignatureDataSet/training"
test_path = "/content/drive/MyDrive/SignatureDataSet/testing"

In [5]:
#Pass train and test images into list and resize to 200 by 200
x_train = []
for folder in os.listdir(train_path):
    sub_path = train_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path = sub_path+"/"+img
        img_arr = cv2.imread(image_path)
        img_arr = cv2.resize(img_arr,(200,200))
        x_train.append(img_arr)

x_test = []
for folder in os.listdir(test_path):
    sub_path = test_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path = sub_path+"/"+img
        img_arr = cv2.imread(image_path)
        img_arr = cv2.resize(img_arr,(200,200))
        x_test.append(img_arr)

In [6]:
#Normalization
train_x = np.array(x_train)
test_x = np.array(x_test)

In [7]:
#Normalization
train_x = train_x/255.0
test_x = test_x/255.0

In [8]:
#Rescaling
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
#Store train and test folder into variables, size 200 by 200, 2 classes
training_set = train_datagen.flow_from_directory(train_path, target_size = (200, 200), batch_size = 10, class_mode = 'binary')

test_set = test_datagen.flow_from_directory(test_path, target_size = (200, 200), batch_size = 10, class_mode = 'binary')

Found 1920 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [10]:
train_y = training_set.classes
test_y = test_set.classes

In [11]:
#Hyperparameters used for ViT
num_classes = 2
input_shape = (200, 200, 3)
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 10
num_epochs = 10
image_size = 200
patch_size = 6
num_patches = (image_size//patch_size)**2
projection_dim = 64
num_heads = 4
transformer_units = [projection_dim*2, projection_dim]
transformer_layers = 8
mlp_head_units = [2048, 1024]

In [12]:
#Normalization and resizing
data_augmentation = keras.Sequential(
    [layers.Normalization(),
    layers.Resizing(image_size, image_size)],
    name = "data_augmentation",
)
data_augmentation.layers[0].adapt(train_x)

In [13]:
#Multilayer perceptron
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [14]:
#Create patches as layers
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = "VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [15]:
#Patch encoding layer
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim = num_patches, output_dim = projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [16]:
#ViT model
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    patches = Patches(patch_size)(augmented)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    logits = layers.Dense(num_classes)(features)
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [17]:
#Compile, train, and evaluate model with test dataset
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

    model.compile(optimizer=optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")])

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor="val_accuracy", save_best_only=True, save_weights_only=True)

    history = model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=num_epochs, validation_split=0.2, callbacks=[checkpoint_callback])

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(test_x, test_y)
    print(f"Overall test accuracy: {round(accuracy * 100, 2)}%")

    return history

vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

Epoch 1/10
154/154 [==============================] - 2970s 19s/step - loss: 16.0066 - accuracy: 0.5378 - val_loss: 3.4466 - val_accuracy: 0.1875
Epoch 2/10
154/154 [==============================] - 3088s 20s/step - loss: 1.4102 - accuracy: 0.6022 - val_loss: 1.2103 - val_accuracy: 0.0234
Epoch 3/10
154/154 [==============================] - 3753s 24s/step - loss: 0.8279 - accuracy: 0.6087 - val_loss: 1.0450 - val_accuracy: 0.0495
Epoch 4/10
154/154 [==============================] - 3452s 22s/step - loss: 0.7013 - accuracy: 0.6374 - val_loss: 0.6455 - val_accuracy: 0.7344
Epoch 5/10
154/154 [==============================] - 3479s 23s/step - loss: 0.6309 - accuracy: 0.6680 - val_loss: 1.0955 - val_accuracy: 0.0286
Epoch 6/10
154/154 [==============================] - 3407s 22s/step - loss: 0.5778 - accuracy: 0.7012 - val_loss: 1.3437 - val_accuracy: 0.1172
Epoch 7/10
154/154 [==============================] - 3448s 22s/step - loss: 0.5497 - accuracy: 0.7298 - val_loss: 1.0809 - val_a